In [1]:
!pip install torchmetrics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 806.1/806.1 kB 5.9 MB/s eta 0:00:00


In [2]:
from google.colab import drive
drive.mount('/content/drive')

import sys
sys.path.append('/content/drive/MyDrive/Colab Notebooks/인공지능프로젝트2/Pyraformer')

from typing import Callable, List, Dict, Union, Optional
import numpy as np
import pandas as pd
import os
from datetime import datetime

import torch
from torch import nn
from torch import Tensor
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, TensorDataset

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

from torchmetrics.functional.regression import spearman_corrcoef
from torchmetrics.regression import PearsonCorrCoef

import argparse
from tqdm import tqdm
from tools import TopkMSELoss, metric
from typing import List
from pandas.tseries import offsets
from pandas.tseries.frequencies import to_offset

import warnings
warnings.filterwarnings('ignore')
device = torch.device('cuda:0')

import torch
import torch.nn as nn
from Layers import EncoderLayer, Decoder, Predictor
from Layers import Bottleneck_Construct, Conv_Construct, MaxPooling_Construct, AvgPooling_Construct
from Layers import get_mask, get_subsequent_mask, refer_points, get_k_q, get_q_k
from embed import DataEmbedding, CustomEmbedding

Mounted at /content/drive


In [3]:
def read_wsdm23_file(file_path: str,) -> Dict[str, pd.DataFrame]:
  wsdm23_dict = np.load(file_path, allow_pickle=True).item()
  for df in wsdm23_dict.values():
    df.index.name = 'date'
  return wsdm23_dict

def load_dataset():
  feature_names = None
  label_names = None

  file_path = '/content/drive/MyDrive/Colab Notebooks/인공지능프로젝트2/baseline_data_sp500.npy'
  df = read_wsdm23_file(file_path)
  return df

df = load_dataset()

In [4]:
dataset = df.copy()
df_list = []
for key in dataset.keys():
  df_list.append(pd.DataFrame({'date':dataset[key].index, 'close':dataset[key]['close'].values, 'open':dataset[key]['open'].values, 'high':dataset[key]['high'].values,\
                               'low':dataset[key]['low'].values, 'volume':dataset[key]['volume'].values, 'adjclose':dataset[key]['adjclose'].values}))
dataset = pd.concat(df_list, ignore_index=True)
dataset['date'] = pd.to_datetime(dataset['date'])
dataset

,date,close,open,high,low,volume,adjclose
0,2012-05-14,36.299999,36.360001,36.840000,35.979999,230900.0,32.237789
1,2012-05-15,36.750000,36.139999,37.119999,36.139999,328500.0,32.637432
2,2012-05-16,36.619999,36.950001,37.080002,36.549999,220300.0,32.521984
3,2012-05-17,34.979999,36.540001,36.540001,34.830002,418400.0,31.065506
4,2012-05-18,35.139999,34.900002,35.660000,34.779999,200300.0,31.207592
...,...,...,...,...,...,...,...
1197319,2022-05-19,90.250000,91.379997,92.459999,89.540001,5234700.0,90.250000
1197320,2022-05-20,90.080002,90.839996,91.169998,88.430000,6585500.0,90.080002
1197321,2022-05-23,91.830002,90.599998,92.019997,90.000000,4701300.0,91.830002
1197322,2022-05-24,93.209999,91.199997,93.419998,90.470001,5932000.0,93.209999


In [5]:
train_start = datetime.strptime('2012-05-14', '%Y-%m-%d')
valid_start = datetime.strptime('2020-05-22', '%Y-%m-%d')
test_start = datetime.strptime('2021-05-22', '%Y-%m-%d')
test_end = datetime.strptime('2022-05-25', '%Y-%m-%d')

train_dataset = dataset[(dataset['date'] >= train_start) & (dataset['date'] < valid_start)]
valid_dataset = dataset[(dataset['date'] >= valid_start) & (dataset['date'] < test_start)]
test_dataset = dataset[(dataset['date'] >= test_start) & (dataset['date'] <= test_end)]

In [6]:
class Encoder(nn.Module):
  """ A encoder model with self attention mechanism. """

  def __init__(self, opt):
    super().__init__()

    self.d_model = opt.d_model
    self.model_type = opt.model
    self.window_size = opt.window_size
    self.truncate = opt.truncate
    if opt.decoder == 'attention':
      self.mask, self.all_size = get_mask(opt.input_size, opt.window_size, opt.inner_size, opt.device)
    else:
      self.mask, self.all_size = get_mask(opt.input_size+1, opt.window_size, opt.inner_size, opt.device)
    self.decoder_type = opt.decoder
    if opt.decoder == 'FC':
      self.indexes = refer_points(self.all_size, opt.window_size, opt.device)

    if opt.use_tvm:
      assert len(set(self.window_size)) == 1, "Only constant window size is supported."
      padding = 1 if opt.decoder == 'FC' else 0
      q_k_mask = get_q_k(opt.input_size + padding, opt.inner_size, opt.window_size[0], opt.device)
      k_q_mask = get_k_q(q_k_mask)
      self.layers = nn.ModuleList([
        EncoderLayer(opt.d_model, opt.d_inner_hid, opt.n_head, opt.d_k, opt.d_v, dropout=opt.dropout, \
            normalize_before=False, use_tvm=True, q_k_mask=q_k_mask, k_q_mask=k_q_mask) for i in range(opt.n_layer)
        ])
    else:
      self.layers = nn.ModuleList([
        EncoderLayer(opt.d_model, opt.d_inner_hid, opt.n_head, opt.d_k, opt.d_v, dropout=opt.dropout, \
            normalize_before=False) for i in range(opt.n_layer)
        ])

    if opt.embed_type == 'CustomEmbedding':
      self.enc_embedding = CustomEmbedding(opt.enc_in, opt.d_model, opt.covariate_size, opt.seq_num, opt.dropout)
    else:
      self.enc_embedding = DataEmbedding(opt.enc_in, opt.d_model, opt.dropout)

    self.conv_layers = eval(opt.CSCM)(opt.d_model, opt.window_size, opt.d_bottleneck)

  def forward(self, x_enc, x_mark_enc):

    seq_enc = self.enc_embedding(x_enc, x_mark_enc)

    mask = self.mask.repeat(len(seq_enc), 1, 1).to(x_enc.device)
    seq_enc = self.conv_layers(seq_enc)

    for i in range(len(self.layers)):
      seq_enc, _ = self.layers[i](seq_enc, mask)

    if self.decoder_type == 'FC':
      indexes = self.indexes.repeat(seq_enc.size(0), 1, 1, seq_enc.size(2)).to(seq_enc.device)
      indexes = indexes.view(seq_enc.size(0), -1, seq_enc.size(2))
      all_enc = torch.gather(seq_enc, 1, indexes)
      seq_enc = all_enc.view(seq_enc.size(0), self.all_size[0], -1)
    elif self.decoder_type == 'attention' and self.truncate:
      seq_enc = seq_enc[:, :self.all_size[0]]

    return seq_enc


class Model(nn.Module):
  """ A sequence to sequence model with attention mechanism. """

  def __init__(self, opt):
    super().__init__()

    self.predict_step = opt.predict_step
    self.d_model = opt.d_model
    self.input_size = opt.input_size
    self.decoder_type = opt.decoder
    self.channels = opt.enc_in

    self.encoder = Encoder(opt)
    if opt.decoder == 'attention':
      mask = get_subsequent_mask(opt.input_size, opt.window_size, opt.predict_step, opt.truncate)
      self.decoder = Decoder(opt, mask)
      self.predictor = Predictor(opt.d_model, opt.enc_in)
    elif opt.decoder == 'FC':
      self.predictor = Predictor(4 * opt.d_model, opt.predict_step * opt.enc_in)

  def forward(self, x_enc, x_mark_enc, x_dec, x_mark_dec, pretrain):
    """
    Return the hidden representations and predictions.
    For a sequence (l_1, l_2, ..., l_N), we predict (l_2, ..., l_N, l_{N+1}).
    Input: event_type: batch*seq_len;
            event_time: batch*seq_len.
    Output: enc_output: batch*seq_len*model_dim;
            type_prediction: batch*seq_len*num_classes (not normalized);
            time_prediction: batch*seq_len.
    """
    if self.decoder_type == 'attention':
      enc_output = self.encoder(x_enc, x_mark_enc)
      dec_enc = self.decoder(x_dec, x_mark_dec, enc_output)

      if pretrain:
        dec_enc = torch.cat([enc_output[:, :self.input_size], dec_enc], dim=1)
        pred = self.predictor(dec_enc)
      else:
        pred = self.predictor(dec_enc)
    elif self.decoder_type == 'FC':
      enc_output = self.encoder(x_enc, x_mark_enc)[:, -1, :]
      pred = self.predictor(enc_output).view(enc_output.size(0), self.predict_step, -1)

    return pred

In [7]:
def sample_mining_scheduler(epoch, batch_size):
  if epoch < 2:
    topk = batch_size
  elif epoch < 4:
    topk = int(batch_size * (5 - epoch) / (6 - epoch))
  else:
    topk = int(0.5 * batch_size)

  return topk

In [8]:
class TimeFeature:
  def __init__(self):
    pass

  def __call__(self, index: pd.DatetimeIndex) -> np.ndarray:
    pass

  def __repr__(self):
    return self.__class__.__name__ + "()"

class SecondOfMinute(TimeFeature):
  """Minute of hour encoded as value between [-0.5, 0.5]"""
  def __call__(self, index: pd.DatetimeIndex) -> np.ndarray:
    return index.second / 59.0 - 0.5

class MinuteOfHour(TimeFeature):
  """Minute of hour encoded as value between [-0.5, 0.5]"""
  def __call__(self, index: pd.DatetimeIndex) -> np.ndarray:
    return index.minute / 59.0 - 0.5

class HourOfDay(TimeFeature):
  """Hour of day encoded as value between [-0.5, 0.5]"""
  def __call__(self, index: pd.DatetimeIndex) -> np.ndarray:
    return index.hour / 23.0 - 0.5

class DayOfWeek(TimeFeature):
  """Hour of day encoded as value between [-0.5, 0.5]"""
  def __call__(self, index: pd.DatetimeIndex) -> np.ndarray:
    return index.dayofweek / 6.0 - 0.5

class DayOfMonth(TimeFeature):
  """Day of month encoded as value between [-0.5, 0.5]"""
  def __call__(self, index: pd.DatetimeIndex) -> np.ndarray:
    return (index.day - 1) / 30.0 - 0.5

class DayOfYear(TimeFeature):
  """Day of year encoded as value between [-0.5, 0.5]"""
  def __call__(self, index: pd.DatetimeIndex) -> np.ndarray:
    return (index.dayofyear - 1) / 365.0 - 0.5

class MonthOfYear(TimeFeature):
  """Month of year encoded as value between [-0.5, 0.5]"""
  def __call__(self, index: pd.DatetimeIndex) -> np.ndarray:
    return (index.month - 1) / 11.0 - 0.5

class WeekOfYear(TimeFeature):
  """Week of year encoded as value between [-0.5, 0.5]"""
  def __call__(self, index: pd.DatetimeIndex) -> np.ndarray:
    return (index.isocalendar().week - 1) / 52.0 - 0.5

def time_features_from_frequency_str(freq_str: str) -> List[TimeFeature]:
  """
  Returns a list of time features that will be appropriate for the given frequency string.
  Parameters
  ----------
  freq_str
      Frequency string of the form [multiple][granularity] such as "12H", "5min", "1D" etc.
  """

  features_by_offsets = {
    offsets.YearEnd: [],
    offsets.QuarterEnd: [MonthOfYear],
    offsets.MonthEnd: [MonthOfYear],
    offsets.Week: [DayOfMonth, WeekOfYear],
    offsets.Day: [DayOfWeek, DayOfMonth, DayOfYear],
    offsets.BusinessDay: [DayOfWeek, DayOfMonth, DayOfYear],
    offsets.Hour: [HourOfDay, DayOfWeek, DayOfMonth, DayOfYear],
    offsets.Minute: [
        MinuteOfHour,
        HourOfDay,
        DayOfWeek,
        DayOfMonth,
        DayOfYear,
    ],
    offsets.Second: [
        SecondOfMinute,
        MinuteOfHour,
        HourOfDay,
        DayOfWeek,
        DayOfMonth,
        DayOfYear,
    ],
  }

  offset = to_offset(freq_str)

  for offset_type, feature_classes in features_by_offsets.items():
    if isinstance(offset, offset_type):
      return [cls() for cls in feature_classes]

  supported_freq_msg = f"""
  Unsupported frequency {freq_str}
  The following frequencies are supported:
    Y   - yearly
        alias: A
    M   - monthly
    W   - weekly
    D   - daily
    B   - business days
    H   - hourly
    T   - minutely
        alias: min
    S   - secondly
  """
  raise RuntimeError(supported_freq_msg)

def time_features(dates, timeenc=1, freq='h'):
  if timeenc==0:
    dates['month'] = dates.date.apply(lambda row:row.month,1)
    dates['day'] = dates.date.apply(lambda row:row.day,1)
    dates['weekday'] = dates.date.apply(lambda row:row.weekday(),1)
    dates['hour'] = dates.date.apply(lambda row:row.hour,1)
    dates['minute'] = dates.date.apply(lambda row:row.minute,1)
    dates['minute'] = dates.minute.map(lambda x:x//15)
    freq_map = {
        'y':[],'m':['month'],'w':['month'],'d':['month','day','weekday'],
        'b':['month','day','weekday'],'h':['month','day','weekday','hour'],
        't':['month','day','weekday','hour','minute'],
    }
    return dates[freq_map[freq.lower()]].values
  if timeenc==1:
    dates = pd.to_datetime(dates.date.values)
    return np.vstack([feat(dates) for feat in time_features_from_frequency_str(freq)]).transpose(1,0)

class SP500(Dataset):
  def __init__(self, flag='train', size=None, inverse=False, dataset=dataset):
    if size == None:
      self.seq_len = 96
      self.pred_len = 5
    else:
      self.seq_len = size[0]
      self.pred_len = size[1]
    assert flag in ['train', 'test', 'val']
    type_map = {'train':0, 'val':1, 'test':2}
    self.set_type = type_map[flag]

    self.inverse = inverse

    self.__read_data__()

  def __read_data__(self):
    self.scaler = StandardScaler()
    df_raw = dataset
    border1s = [0, 12*30*24 - self.seq_len, 12*30*24+4*30*24 - self.seq_len]
    border2s = [12*30*24, 12*30*24+4*30*24, 12*30*24+8*30*24]
    border1 = border1s[self.set_type]
    border2 = border2s[self.set_type]

    cols_data = df_raw.columns[1:]
    df_data = df_raw[cols_data]

    train_data = df_data[border1s[0]:border2s[0]]
    self.scaler.fit(train_data.values)
    data = self.scaler.transform(df_data.values)

    df_stamp = df_raw[['date']][border1:border2]
    df_stamp['date'] = pd.to_datetime(df_stamp.date)
    data_stamp = time_features(df_stamp, timeenc=1, freq='h')

    self.data_x = data[border1:border2]
    if self.inverse:
      self.data_y = df_data.values[border1:border2]
    else:
      self.data_y = data[border1:border2]
    self.data_stamp = data_stamp

  def __getitem__(self, index):
    s_begin = index
    s_end = s_begin + self.seq_len
    r_begin = s_end
    r_end = r_begin + self.pred_len

    seq_x = self.data_x[s_begin:s_end]
    seq_y = self.data_y[r_begin:r_end]
    seq_x_mark = self.data_stamp[s_begin:s_end]
    seq_y_mark = self.data_stamp[r_begin:r_end]

    return seq_x, seq_y, seq_x_mark, seq_y_mark, self.scaler.mean_, np.sqrt(self.scaler.var_)

  def __len__(self):
    return len(self.data_x) - self.seq_len - self.pred_len + 1

  def inverse_transform(self, data, seq_y, mean, std):
    return self.scaler.inverse_transform(data), seq_y

In [9]:
def prepare_dataloader(args):
  Data = SP500
  drop_last = True; batch_size = args.batch_size
  train_set = Data(flag='train', size=[args.input_size, args.predict_step], inverse=args.inverse, dataset=train_dataset)
  train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True, num_workers=0, drop_last=drop_last)

  drop_last = True; batch_size = args.batch_size
  val_set = Data(flag='val', size=[args.input_size, args.predict_step], inverse=args.inverse, dataset=valid_dataset)
  val_loader = DataLoader(val_set, batch_size=batch_size, shuffle=True, num_workers=0, drop_last=drop_last)

  drop_last = False; batch_size = args.batch_size
  test_set = Data(flag='test', size=[args.input_size, args.predict_step], inverse=args.inverse, dataset=test_dataset)
  test_loader = DataLoader(test_set, batch_size=batch_size, shuffle=False, num_workers=0, drop_last=drop_last)

  return train_loader, train_set, val_loader, val_set, test_loader, test_set

In [10]:
def train_epoch(model, train_dataset, training_loader, optimizer, opt, epoch):
  """ Epoch operation in training phase. """

  model.train()
  total_loss = 0
  total_pred_number = 0
  for batch in tqdm(training_loader, mininterval=2, desc='  - (Training)   ', leave=False):
    # prepare data
    batch_x, batch_y, batch_x_mark, batch_y_mark, mean, std = map(lambda x: x.float().to(opt.device), batch)
    # prepare predict token
    dec_inp = torch.zeros_like(batch_y).float()

    optimizer.zero_grad()

    # forward
    if opt.decoder == 'attention':
      if opt.pretrain and epoch < 1:
        outputs = model(batch_x, batch_x_mark, dec_inp, batch_y_mark, True)
        batch_y = torch.cat([batch_x, batch_y], dim=1)
      else:
        outputs = model(batch_x, batch_x_mark, dec_inp, batch_y_mark, False)
    elif opt.decoder == 'FC':
      # Add a predict token into the history sequence
      predict_token = torch.zeros(batch_x.size(0), 1, batch_x.size(-1), device=batch_x.device)
      batch_x = torch.cat([batch_x, predict_token], dim=1)
      batch_x_mark = torch.cat([batch_x_mark, batch_y_mark[:, 0:1, :]], dim=1)
      outputs = model(batch_x, batch_x_mark, dec_inp, batch_y_mark, False)

    # determine the loss function
    if opt.hard_sample_mining and not (opt.pretrain and epoch < 1):
      topk = sample_mining_scheduler(epoch, batch_x.size(0))
      criterion = TopkMSELoss(topk)
    else:
      criterion = torch.nn.MSELoss(reduction='none')
    # if inverse, both the output and the ground truth are denormalized.
    if opt.inverse:
      outputs, batch_y = train_dataset.inverse_transform(outputs, batch_y, mean, std)
    # compute loss
    losses = criterion(outputs, batch_y)
    loss = losses.mean()
    loss.backward()

    """ update parameters """
    optimizer.step()
    total_loss += losses.sum().item()
    total_pred_number += losses.numel()

  return total_loss / total_pred_number

In [11]:
def eval_epoch(model, test_dataset, test_loader, opt, epoch):
  """ Epoch operation in evaluation phase. """

  model.eval()
  preds = []
  trues = []
  with torch.no_grad():
    for batch in tqdm(test_loader, mininterval=2, desc='  - (Validation) ', leave=False):
      """ prepare data """
      batch_x, batch_y, batch_x_mark, batch_y_mark, mean, std = map(lambda x: x.float().to(opt.device), batch)
      dec_inp = torch.zeros_like(batch_y).float()

      # forward
      if opt.decoder == 'FC':
        # Add a predict token into the history sequence
        predict_token = torch.zeros(batch_x.size(0), 1, batch_x.size(-1), device=batch_x.device)
        batch_x = torch.cat([batch_x, predict_token], dim=1)
        batch_x_mark = torch.cat([batch_x_mark, batch_y_mark[:, 0:1, :]], dim=1)
      outputs = model(batch_x, batch_x_mark, dec_inp, batch_y_mark, False)

      # if inverse, both the output and the ground truth are denormalized.
      if opt.inverse:
        outputs, batch_y = test_dataset.inverse_transform(outputs, batch_y, mean, std)

      pred = outputs.detach().cpu().numpy()
      true = batch_y.detach().cpu().numpy()

      preds.append(pred)
      trues.append(true)

  preds = np.concatenate(preds, axis=0)
  trues = np.concatenate(trues, axis=0)
  print('test shape:{}'.format(preds.shape))
  mae, mse, rmse, mape, mspe = metric(preds, trues)
  print('Epoch {}, mse:{}, mae:{}, rmse:{}'.format(epoch, mse, mae, rmse))

  return mse, mae, rmse, preds, trues

In [12]:
def train(model, optimizer, scheduler, opt):
  best_mse = 10000000
  training_dataloader, train_dataset, val_dataloader, val_dataset, _, _ = prepare_dataloader(opt)
  best_metrics = []
  for epoch_i in range(opt.epoch):
    epoch = epoch_i + 1
    print('[ Epoch', epoch, ']')

    train_mse = train_epoch(model, train_dataset, training_dataloader, optimizer, opt, epoch_i)
    print('  - (Training) '
          'MSE: {mse: 8.5f}'.format(mse=train_mse))

    mse, mae, rmse, preds, trues = eval_epoch(model, val_dataset, val_dataloader, opt, epoch_i)

    scheduler.step()

    current_metrics = [float(mse), float(mae), float(rmse)]
    if best_mse > mse:
      best_mse = mse
      best_metrics = current_metrics
#      torch.save(
#        {
#          "state_dict": model.state_dict(),
#          "metrics": best_metrics
#        },
#        model_save_dir
#      )

  return best_metrics, preds, trues

In [13]:
def evaluate(model, opt):
  """Evaluate preptrained models"""
  best_mse = 100000000

  """ prepare dataloader """
  _, _, _, _, test_dataloader, test_dataset = prepare_dataloader(opt)

  """ load pretrained model """
#  checkpoint = torch.load(model_save_dir)["state_dict"]
#  model.load_state_dict(checkpoint)

  best_metrics = []
  mse, mae, rmse, _, _ = eval_epoch(model, test_dataset, test_dataloader, opt, 0)

  current_metrics = [float(mse), float(mae), float(rmse)]
  if best_mse > mse:
    best_mse = mse
    best_metrics = current_metrics

  return best_metrics

In [14]:
parser = argparse.ArgumentParser()

# running mode
parser.add_argument('-eval', action='store_true', default=False)
parser.add_argument('-device', default=device)

# Dataloader parameters.
parser.add_argument('-input_size', type=int, default=96)
parser.add_argument('-predict_step', type=int, default=5)
parser.add_argument('-inverse', action='store_true', help='denormalize output data', default=False)
parser.add_argument('-data', default=dataset)

# Architecture selection.
parser.add_argument('-model', type=str, default='Model')
parser.add_argument('-decoder', type=str, default='FC') # selection: [FC, attention]
parser.add_argument('-embed_type', type=str, default='DataEmbedding')

# Training parameters.
parser.add_argument('-epoch', type=int, default=5)
parser.add_argument('-batch_size', type=int, default=32)
parser.add_argument('-pretrain', action='store_true', default=False)
parser.add_argument('-hard_sample_mining', action='store_true', default=False)
parser.add_argument('-dropout', type=float, default=0.05)
parser.add_argument('-lr', type=float, default=1e-4)
parser.add_argument('-lr_step', type=float, default=0.1)

# Common Model parameters.
parser.add_argument('-d_model', type=int, default=512)
parser.add_argument('-d_inner_hid', type=int, default=512)
parser.add_argument('-d_k', type=int, default=128)
parser.add_argument('-d_v', type=int, default=128)
parser.add_argument('-d_bottleneck', type=int, default=128)
parser.add_argument('-n_head', type=int, default=4)
parser.add_argument('-n_layer', type=int, default=4)
parser.add_argument('-enc_in', type=int, default=6)

# Pyraformer parameters.
parser.add_argument('-window_size', type=str, default='[4, 4, 4]') # The number of children of a parent node.
parser.add_argument('-inner_size', type=int, default=3) # The number of ajacent nodes.

# CSCM structure. selection: [Bottleneck_Construct, Conv_Construct, MaxPooling_Construct, AvgPooling_Construct]
parser.add_argument('-CSCM', type=str, default='Bottleneck_Construct')
parser.add_argument('-truncate', action='store_true', default=False) # Whether to remove coarse-scale nodes from the attention structure
parser.add_argument('-use_tvm', action='store_true', default=False) # Whether to use TVM.

# Experiment repeat times.
parser.add_argument('-iter_num', type=int, default=5) # Repeat number.

opt = parser.parse_args('')

In [15]:
def main(opt, iter_index):
  """ Main function. """
  print('[Info] parameters: {}'.format(opt))

  if torch.cuda.is_available():
    opt.device = torch.device("cuda")
  else:
    opt.device = torch.device('cpu')

  """ prepare model """
  model = Model(opt).to(device)

  """ number of parameters """
  num_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
  print('[Info] Number of parameters: {}'.format(num_params))

  """ train or evaluate the model """
 # model_save_dir = './predict{}/'.format(opt.predict_step)
 # os.makedirs(model_save_dir, exist_ok=True)
 # model_save_dir = os.path.join(model_save_dir,'best_iter{}.pth'.format(iter_index))
  if opt.eval:
    best_metrics = evaluate(model, opt)

  else:
    """ optimizer and scheduler """
    optimizer = optim.Adam(filter(lambda x: x.requires_grad, model.parameters()), opt.lr)
    scheduler = optim.lr_scheduler.StepLR(optimizer, 1, gamma=opt.lr_step)
    best_metrics, preds, trues = train(model, optimizer, scheduler, opt)

    preds = torch.tensor(preds, dtype=torch.float32)
    trues = torch.tensor(trues, dtype=torch.float32)
    preds = preds.reshape((85440,1)).cpu()
    trues = trues.reshape((85440,1)).cpu()
    rankIC = spearman_corrcoef(preds, trues)
    print('RankIC: {}'.format(rankIC))
    rankIR = PearsonCorrCoef()
    rankIR(preds, trues)
    print('RankIR: {}'.format(rankIR.compute()))

  print('Iteration best metrics: {}'.format(best_metrics))
  return best_metrics

In [16]:
if __name__ == '__main__':
  opt.window_size = eval(opt.window_size)
  iter_num = opt.iter_num
  all_perf = []
  for i in range(iter_num):
    metrics = main(opt, i)
    all_perf.append(metrics)
  all_perf = np.array(all_perf)
  all_perf = all_perf.mean(0)
  print('Average Metrics: {}'.format(all_perf))

[Info] parameters: Namespace(eval=False, device=device(type='cuda', index=0), input_size=96, predict_step=5, inverse=False, data=              date      close       open       high        low     volume  \
0       2012-05-14  36.299999  36.360001  36.840000  35.979999   230900.0   
1       2012-05-15  36.750000  36.139999  37.119999  36.139999   328500.0   
2       2012-05-16  36.619999  36.950001  37.080002  36.549999   220300.0   
3       2012-05-17  34.979999  36.540001  36.540001  34.830002   418400.0   
4       2012-05-18  35.139999  34.900002  35.660000  34.779999   200300.0   
...            ...        ...        ...        ...        ...        ...   
1197319 2022-05-19  90.250000  91.379997  92.459999  89.540001  5234700.0   
1197320 2022-05-20  90.080002  90.839996  91.169998  88.430000  6585500.0   
1197321 2022-05-23  91.830002  90.599998  92.019997  90.000000  4701300.0   
1197322 2022-05-24  93.209999  91.199997  93.419998  90.470001  5932000.0   
1197323 2022-05-25  93.5

  - (Training) MSE:  0.23059


test shape:(2848, 5, 6)
Epoch 0, mse:0.10844556242227554, mae:0.1733916848897934, rmse:0.32931074500083923
[ Epoch 2 ]


  - (Training) MSE:  0.12892


test shape:(2848, 5, 6)
Epoch 1, mse:0.09018637239933014, mae:0.1436559408903122, rmse:0.3003104627132416
[ Epoch 3 ]


...

test shape:(2848, 5, 6)
Epoch 2, mse:0.08741501718759537, mae:0.13803254067897797, rmse:0.2956603169441223
[ Epoch 4 ]


  - (Training) MSE:  0.12715


test shape:(2848, 5, 6)
Epoch 3, mse:0.08562524616718292, mae:0.13531440496444702, rmse:0.29261791706085205
[ Epoch 5 ]


  - (Training) MSE:  0.12546


test shape:(2848, 5, 6)
Epoch 4, mse:0.08464173972606659, mae:0.13423575460910797, rmse:0.2909325361251831
RankIC: tensor([0.9223])
RankIR: 0.8467590808868408
Iteration best metrics: [0.08464173972606659, 0.13423575460910797, 0.2909325361251831]
Average Metrics: [0.08737865 0.13853184 0.29557098]
